In [2]:
import sys
sys.path.append('../src')
import json

with open('../data/output_withStateId.json') as f:
    json_data = json.load(f)

In [3]:
from dataframeFactory import get_data_frame

data = get_data_frame(json_data)

In [4]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform
import numpy as np
from sklearn.gaussian_process.kernels import Matern

# Separate data into observations and missing values
observations = data[data['all_measurements']!=0]
missing = data[data['all_measurements']==0]



# Split the observations into a training set and a test set
train, test = train_test_split(observations, test_size=0.4, random_state=42)
train = train.to_crs("EPSG:4326")
missing = missing.to_crs("EPSG:4326")
test = test.to_crs("EPSG:4326")
if len(train) > 0 and len(missing) > 0:
    # Fit a Gaussian Process Regressor on the training data
    X_train = train['geometry'].centroid.apply(lambda point: [point.x, point.y])
    X_train = np.array(X_train.tolist())
    y_train = train['all_stability']

    # Define the model and the parameter distribution for the random search
    gpr = GaussianProcessRegressor(kernel=Matern())
    param_dist = {'alpha': uniform(1e-3, 1), 'kernel__length_scale': uniform(1, 10), 'kernel__nu': uniform(0.5, 2.5)}

    # Perform the random search
    random_search = RandomizedSearchCV(gpr, param_distributions=param_dist, n_iter=10, cv=5, random_state=42)
    random_search.fit(X_train, y_train)

    # Print the best parameters
    # Print the best parameters
    print(f'Best parameters:\nAlpha: {random_search.best_params_["alpha"]:.6f}\nLength Scale: {random_search.best_params_["kernel__length_scale"]:.6f}\n')
    best_index = random_search.best_index_
    best_mse = -random_search.cv_results_['mean_test_score'][best_index]
    best_std = random_search.cv_results_['std_test_score'][best_index]
    print(f'MSE for best parameters: {best_mse:.6f}\nStd: {best_std:.6f}\n')
    # Print all hyperparameters and their corresponding mean squared errors
    for params, mean_score, scores in zip(random_search.cv_results_['params'], 
                                        random_search.cv_results_['mean_test_score'], 
                                        random_search.cv_results_['std_test_score']):
        print(f'Parameters:\nAlpha: {params["alpha"]:.6f}\nLength Scale: {params["kernel__length_scale"]:.6f}\nMean MSE: {-mean_score:.6f}\nStd: {scores:.6f}\n')

    # Calculate the MSE on the test set
    X_test_actual = test['geometry'].centroid.apply(lambda point: [point.x, point.y])
    X_test_actual = np.array(X_test_actual.tolist())
    y_test_actual = test['all_stability']
    y_pred_actual = random_search.predict(X_test_actual)
    mse = mean_squared_error(y_test_actual, y_pred_actual)
    print(f'Mean Squared Error on Test Set: {mse:.6f}')

/var/folders/_v/2pyg9ssn20xcgcmlm6srjwh40000gn/T/ipykernel_20759/2213108763.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  X_train = train['geometry'].centroid.apply(lambda point: [point.x, point.y])
/Users/jonasfluck/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/jonasfluck/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/jonasfluck/a

Best parameters:
Alpha: 0.515234
Length Scale: 6.924146

MSE for best parameters: 0.000659
Std: 0.000443

Parameters:
Alpha: 0.375540
Length Scale: 10.507143
Mean MSE: 0.000768
Std: 0.000439

Parameters:
Alpha: 0.599658
Length Scale: 2.560186
Mean MSE: 0.000770
Std: 0.000458

Parameters:
Alpha: 0.059084
Length Scale: 9.661761
Mean MSE: 0.000765
Std: 0.000413

Parameters:
Alpha: 0.709073
Length Scale: 1.205845
Mean MSE: 0.000771
Std: 0.000467

Parameters:
Alpha: 0.833443
Length Scale: 3.123391
Mean MSE: 0.000773
Std: 0.000477

Parameters:
Alpha: 0.184405
Length Scale: 4.042422
Mean MSE: 0.000766
Std: 0.000423

Parameters:
Alpha: 0.432945
Length Scale: 3.912291
Mean MSE: 0.000769
Std: 0.000444

Parameters:
Alpha: 0.140494
Length Scale: 3.921446
Mean MSE: 0.000766
Std: 0.000419

Parameters:
Alpha: 0.457070
Length Scale: 8.851760
Mean MSE: 0.000769
Std: 0.000446

Parameters:
Alpha: 0.515234
Length Scale: 6.924146
Mean MSE: 0.000659
Std: 0.000443



/var/folders/_v/2pyg9ssn20xcgcmlm6srjwh40000gn/T/ipykernel_20759/2213108763.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  X_test_actual = test['geometry'].centroid.apply(lambda point: [point.x, point.y])


Mean Squared Error on Test Set: 0.007286


In [5]:
import pandas as pd


alpha = random_search.best_params_["alpha"]
length_scale = random_search.best_params_["kernel__length_scale"]
nu = random_search.best_params_["kernel__nu"]

data['uncertainty'] = None

# Fit the model on the training data
observations = data[data['all_measurements']!=0]
missing = data[data['all_measurements']==0]

observations = observations.to_crs("EPSG:4326")
missing = missing.to_crs("EPSG:4326")

data['uncertainty'] = None  # Initialize 'uncertainty' with a default value

std_devs = None
if len(observations) > 0 and len(missing) > 0:
    # Fit a Gaussian Process Regressor on the observed data
    X_train = observations['geometry'].centroid.apply(lambda point: [point.x, point.y])
    X_train = np.array(X_train.tolist())
    y_train = observations['all_stability']

    gpr = GaussianProcessRegressor(alpha=alpha, kernel=Matern(length_scale=length_scale, nu = nu)).fit(X_train, y_train)

    # Predict the missing values and get standard deviations
    X_test = missing['geometry'].centroid.apply(lambda point: [point.x, point.y])
    X_test = np.array(X_test.tolist())
    y_pred, std_devs = gpr.predict(X_test, return_std=True)

    y_pred = np.clip(y_pred, 0, 1)
    data['all_stability'] = data['all_stability'].astype(float)

    # Fill in the missing values
    data.loc[missing.index, 'all_stability'] = y_pred
    data.loc[missing.index, 'uncertainty'] = std_devs


# Convert the data back to JSON




/var/folders/_v/2pyg9ssn20xcgcmlm6srjwh40000gn/T/ipykernel_20759/1741731566.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  X_train = observations['geometry'].centroid.apply(lambda point: [point.x, point.y])
/Users/jonasfluck/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/var/folders/_v/2pyg9ssn20xcgcmlm6srjwh40000gn/T/ipykernel_20759/1741731566.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  X_test = missing['geometry'].centroid.apply(lambda point: [

In [6]:
alpha = random_search.best_params_["alpha"]
length_scale = random_search.best_params_["kernel__length_scale"]
providers = ['t-mobile', 'vodafone', 'o2', 'e-plus']  # List of providers

for provider in providers:
    # Separate data into observations and missing values
    observations = data[data[provider+'_measurements']!=0]
    missing = data[data[provider+'_measurements']==0]
    observations = observations.to_crs("EPSG:4326")
    missing = missing.to_crs("EPSG:4326")

    data[provider+'_uncertainty'] = None  # Initialize 'uncertainty' with a default value

    if len(observations) > 0 and len(missing) > 0:
        # Fit a Gaussian Process Regressor on the observed data
        X_train = observations['geometry'].centroid.apply(lambda point: [point.x, point.y])
        X_train = np.array(X_train.tolist())
        
        y_train = observations[provider+'_stability']
        
        gpr = GaussianProcessRegressor(alpha=alpha, kernel=Matern(length_scale=length_scale, nu=nu)).fit(X_train, y_train)

        X_test = missing['geometry'].centroid.apply(lambda point: [point.x, point.y])
        X_test = np.array(X_test.tolist())

        y_pred, std_devs = gpr.predict(X_test, return_std=True)

        y_pred = np.clip(y_pred, 0, 1)
        data[provider+'_stability'] = data[provider+'_stability'].astype(float)

        # Fill in the missing values
        data.loc[missing.index, provider+'_stability'] = y_pred
        data.loc[missing.index, provider+'_uncertainty'] = std_devs

/var/folders/_v/2pyg9ssn20xcgcmlm6srjwh40000gn/T/ipykernel_20759/3198428514.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  X_train = observations['geometry'].centroid.apply(lambda point: [point.x, point.y])


In [ ]:
json_data = data.to_json(indent=4)
with open('output_matern_60_40.json', 'w') as f:
    f.write(json_data)